In [1]:
# notebooks/data_exploration.ipynb

# Import necessary libraries
import pandas as pd

# Load datasets
application_df = pd.read_csv('./data/raw/application_record.csv')
credit_df = pd.read_csv('./data/raw/credit_record.csv')

# Display first few rows of application data
print(application_df.head())

# Display first few rows of credit data
print(credit_df.head())

# Basic information about datasets
print(application_df.info())
print(credit_df.info())

# Check if 'TARGET' column is in the dataset (you may need to add or rename it)
print("Columns in application_df:", application_df.columns)
print("Columns in credit_df:", credit_df.columns)



        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008809           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3          270000.0  Commercial associate  Secondary / secondary special   
4          270000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0        Civil marriage   Rented apartment      -12005 

In [2]:
# Import necessary libraries
import pandas as pd

# Function to load data
def load_data():
    application_df = pd.read_csv('./data/raw/application_record.csv')
    credit_df = pd.read_csv('./data/raw/credit_record.csv')
    return application_df, credit_df

# Function to preprocess data
def preprocess_data(application_df, credit_df):
    # Drop duplicates
    application_df.drop_duplicates(inplace=True)
    credit_df.drop_duplicates(inplace=True)
    
    # Create 'TARGET' column: 1 if STATUS indicates overdue payments, else 0
    credit_df['TARGET'] = credit_df['STATUS'].apply(lambda x: 1 if x in ['2', '3', '4', '5'] else 0)
    
    # Aggregate 'TARGET' to have one row per ID
    credit_agg = credit_df.groupby('ID')['TARGET'].max().reset_index()
    
    # Merge datasets on ID column
    merged_data = pd.merge(application_df, credit_agg, on='ID', how='inner')
    
    return merged_data

# Load and preprocess data
application_df, credit_df = load_data()
merged_data = preprocess_data(application_df, credit_df)
merged_data.to_csv('../data/processed/merged_data.csv', index=False)

# Display first few rows of merged data
print(merged_data.head())

# Check the columns to ensure 'TARGET' is present
print("Columns in merged_data:", merged_data.columns)


        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008809           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3          270000.0  Commercial associate  Secondary / secondary special   
4          270000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0        Civil marriage   Rented apartment      -12005 

In [3]:
# notebooks/feature_engineering.ipynb

# Import necessary libraries
import pandas as pd

# Load preprocessed data
merged_data = pd.read_csv('../data/processed/merged_data.csv')

# Function for feature engineering
def feature_engineering(merged_data):
    # Feature engineering example
    merged_data['Income_Per_Family_Member'] = merged_data['AMT_INCOME_TOTAL'] / merged_data['CNT_FAM_MEMBERS']
    
    # Encode categorical variables
    categorical_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
                        'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
    merged_data = pd.get_dummies(merged_data, columns=categorical_cols, drop_first=True)
    
    return merged_data

# Apply feature engineering
feature_engineered_data = feature_engineering(merged_data)
feature_engineered_data.to_csv('../data/processed/feature_engineered_data.csv', index=False)

# Display first few rows of feature engineered data
print(feature_engineered_data.head())

# Check the columns to ensure 'TARGET' is present
print("Columns in feature_engineered_data:", feature_engineered_data.columns)


        ID  CNT_CHILDREN  AMT_INCOME_TOTAL  DAYS_BIRTH  DAYS_EMPLOYED  \
0  5008804             0          427500.0      -12005          -4542   
1  5008805             0          427500.0      -12005          -4542   
2  5008806             0          112500.0      -21474          -1134   
3  5008808             0          270000.0      -19110          -3051   
4  5008809             0          270000.0      -19110          -3051   

   FLAG_MOBIL  FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL  CNT_FAM_MEMBERS  ...  \
0           1                1           0           0              2.0  ...   
1           1                1           0           0              2.0  ...   
2           1                0           0           0              2.0  ...   
3           1                0           1           1              1.0  ...   
4           1                0           1           1              1.0  ...   

   OCCUPATION_TYPE_Laborers  OCCUPATION_TYPE_Low-skill Laborers  \
0            

In [4]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load feature engineered data
feature_engineered_data = pd.read_csv('../data/processed/feature_engineered_data.csv')

# Ensure 'TARGET' column is present
if 'TARGET' not in feature_engineered_data.columns:
    raise KeyError("'TARGET' column not found in the dataset")

# Train model function
def train_model(feature_engineered_data):
    X = feature_engineered_data.drop('TARGET', axis=1)
    y = feature_engineered_data['TARGET']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print("Accuracy:", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    
    # Save the model to the manually created models directory
    joblib.dump(model, './models/credit_approval_model.pkl')

# Train the model
train_model(feature_engineered_data)



Accuracy: 0.9786066922654965
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7175
           1       0.28      0.21      0.24       117

    accuracy                           0.98      7292
   macro avg       0.63      0.60      0.62      7292
weighted avg       0.98      0.98      0.98      7292

